# **LoopsResultsAnalysis**

## **1. Importing and First Proccesing**

In [8]:
import pandas as pd
from scipy import stats
import plotly.express as px
from DataCleaning import *
from ProcessingConfig import *
from datetime import datetime as dt

pd.options.display.max_columns = 50

In [9]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])
print(f'original shape: {raw_data.shape}')
outliers_threshold = cleaning_config['filter_threshold']
print(f"threshold for outliers detection: {outliers_threshold}")

drop_columns(raw_data, cleaning_config['unnecessary_columns'])
convert_types(raw_data, cleaning_config['type_conversions'])
raw_data = filter_slow_subjects(raw_data, outliers_threshold)
raw_data = filter_bad_subjects(raw_data, outliers_threshold)
raw_data = drop_first_loop(raw_data)
raw_data = only_first_line(raw_data)
raw_data = filter_bad_trials(raw_data, threshold=0.5)
raw_data = filter_slow_steps(raw_data, outliers_threshold)

path = cleaning_config['results_path'] + f'_{dt.now().strftime("%d.%m.%Y_%H-%M")}.xlsx'
raw_data.to_excel(path)

print(f'final shape: {raw_data.shape}')

In [ ]:
n_subjects = raw_data['subject'].nunique()
print(f'number of subjects: {n_subjects}')

n_sessions = raw_data[['subject', 'trial_set']].drop_duplicates().shape[0]
print(f'number of sessions: {n_sessions}')

number of subjects: 17
number of sessions: 31


## **2. Priming Effect Testing**

In [ ]:
alpha = 0.05

# enveloping t_test and f_test functions

def f_test(smp1, smp2, alpha):
    f_score = smp1.std() / smp2.std()
    df1, df2 = smp1.size - 1, smp2.size - 1
    return stats.f.cdf(f_score, df1, df2) >= alpha

def t_test_ind(smp1, smp2, alpha, alternative='two-sided'):
    equal_var = f_test(smp1, smp2, alpha)
    t_score, p_value = stats.ttest_ind(smp1, smp2, alternative=alternative, equal_var=equal_var)
    
    print(f'p_value: {p_value}')
    if p_value <= alpha:
        print(f'There is a significant difference between the samples! ({(1-alpha)*100}%).')
    else:
        print(f'It is not possible to determine whether there is an effect ({(1-alpha)*100}%).')
    return t_score, p_value

In [ ]:
# mean response time and success rate, group by loop type and loop switching.
switching_diff = pd.DataFrame()

switching_diff['mean_response_time'] = raw_data[raw_data['correct']].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()
switching_diff['mean_success_rate'] = raw_data.groupby(['loop_type', 'loop_type_switch'])['correct'].mean()

switching_diff

mean_response_time  mean_success_rate
loop_type loop_type_switch                                       
for       False                    3280.274038           0.973479
          True                     3265.996753           0.973144
while     False                    3585.572148           0.953600
          True                     3689.630182           0.964800

In [ ]:
# checking for priming effect on 'for' loops
same = raw_data.loc[raw_data['correct'] & ( ~ raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'for'), 'rt']
different = raw_data.loc[raw_data['correct'] & (raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'for'), 'rt']

t_score, p_value = t_test_ind(different, same, alpha=alpha, alternative='greater')

p_value: 0.5576595681879694
It is not possible to determine whether there is an effect (95.0%).


In [ ]:
# checking for priming effect on 'while' loops
same = raw_data.loc[raw_data['correct'] & ( ~ raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'while'), 'rt']
different = raw_data.loc[raw_data['correct'] & (raw_data['loop_type_switch']) & (raw_data['loop_type'] == 'while'), 'rt']

t_score, p_value = t_test_ind(different, same, alpha=alpha, alternative='greater')

p_value: 0.17272443115918634
It is not possible to determine whether there is an effect (95.0%).
